## How to reconstruct exact quasiprobability distributions from Sampler

This how-to guide is intended to show users how they can generate statevector-based quasiprobability distributions through the `qiskit.primitives.BaseSampler` interface.

In [1]:
import numpy as np
from qiskit.circuit.library import EfficientSU2
from qiskit.quantum_info import PauliList
from qiskit_aer.primitives import Estimator, Sampler

from circuit_knitting_toolbox.circuit_cutting import (
    partition_problem,
    execute_experiments,
    reconstruct_expectation_values,
)

Prepare inputs to `execute_experiments`

In [2]:
circuit = EfficientSU2(4, entanglement="linear", reps=2).decompose()
circuit.assign_parameters([0.8] * len(circuit.parameters), inplace=True)
observables = PauliList(["ZZII", "IZZI", "IIZZ", "XIXI", "ZIZZ", "IXIX"])
subcircuits, bases, subobservables = partition_problem(
    circuit=circuit, partition_labels="AABB", observables=observables
)

In order to calculate exact quasiprobability distributions for circuits with mid-circuit measurements, users will need to use the `ExactSampler` class from `circuit_knitting_toolbox.utils.simulation`. The Qiskit Samplers do not support mid-circuit measurements in statevector mode.

In [3]:
from circuit_knitting_toolbox.utils.simulation import ExactSampler

exact_sampler = ExactSampler()

If `ExactSampler` is used, the quasiprobability distributions returned from `execute_experiments` will be exact and generated from the statevectors of the subexperiments.

In [4]:
quasi_dists, coefficients = execute_experiments(
    circuits=subcircuits,
    subobservables=subobservables,
    num_samples=1500,
    samplers=exact_sampler,
)